### Treatment

In [44]:
import pandas as pd
import numpy as np

treatment = pd.read_csv('treatment.csv')
print(treatment)

      No                                     Treatment     item number list
0      1                                  Consultation                  015
1      2              Consultation and TP presentation              015/018
2      3                            Treatment Planning  037.072.073.221.011
3      4                        Quick clean - scalling                  114
4      5                Routine Clean - dental checkup      013.114.121.141
..   ...                                           ...                  ...
116  117                     2 Bridge treatment  1A,1P  71.71.613.643.22.22
117  118                 Provisional crown – per tooth                  631
118  119  Repair of crown, bridge or splint - indirect                  658
119  120    Repair of crown, bridge or splint – direct                  659
120  121                                       Monit 1  14.115.72.73.141.73

[121 rows x 3 columns]


In [45]:
import pandas as pd
import re

# Function to clean the data for each entry in the DataFrame
def clean_data(input_string):
    # Handle empty or missing values
    if pd.isna(input_string):
        return []
    
    # Remove spaces
    input_string = input_string.replace(' ', '')

    # Handle groups like (825/825/071)x3 where each number in the group should be repeated N times
    def handle_group_multiplier(match):
        numbers_in_group = match.group(1)
        repeat_count = int(match.group(2))
        # Split the numbers and repeat them N times
        numbers = re.split(r'[^0-9]+', numbers_in_group)
        return ', '.join(numbers * repeat_count)

    # Replace group multiplier patterns like (825/825/071)x3
    input_string = re.sub(r'\(([^()]+)\)x(\d+)', handle_group_multiplier, input_string)
    
    # Handle individual 'x' multiplier cases inside groups (e.g., '862x6' becomes '862, 862, 862, 862, 862, 862')
    def handle_single_multiplier(match):
        number, count = match.groups()
        return ', '.join([number] * int(count))
    
    # Replace single 'x' patterns (e.g., '862x6')
    input_string = re.sub(r'(\d+)x(\d+)', handle_single_multiplier, input_string)

    # Split numbers by non-digit characters and return a list of strings (to preserve leading zeros)
    numbers = re.split(r'[^0-9]+', input_string)
    return [num.zfill(3) for num in numbers if num]  # Ensure leading zeros are preserved

# # Load the CSV file into a pandas DataFrame
# df = pd.read_csv('your_file.csv')  # Replace 'your_file.csv' with your actual file path

# # Apply the clean_data function to the 'item number list' column
# df['cleaned_item_numbers'] = df['item number list'].apply(clean_data)

# # Output the updated DataFrame
# print(df[['item number list', 'cleaned_item_numbers']])


In [46]:
treatment['cleaned_item_numbers'] = treatment['item number list'].apply(clean_data)
treatment.to_csv('cleaned_treatment.csv', index=False)

In [50]:
treatment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   No                    121 non-null    int64 
 1   Treatment             121 non-null    object
 2   item number list      121 non-null    object
 3   cleaned_item_numbers  121 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.9+ KB


### Item Code

In [57]:
item_code = pd.read_csv('item_code.csv')
item_code

,No,AU Time (Min),Offer Gaia Price,Cost Material
0,11,0:30:00,49.0,15
1,12,0:15:00,33.0,10
2,13,0:15:00,30.0,9
3,14,0:15:00,34.0,10
4,15,0:30:00,83.0,25
...,...,...,...,...
337,986,0:30:00,71.0,21
338,987,0:30:00,67.0,20
339,990,0:45:00,134.0,40
340,999,0:30:00,0.0,0


In [58]:
# make column 'No' to be string and for the leading zeros to be preserved only for values contain only 2 digits
item_code['No'] = item_code['No'].apply(lambda x: str(x).zfill(3) if len(str(x)) == 2 else str(x))
item_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   No                342 non-null    object 
 1   AU Time (Min)     342 non-null    object 
 2   Offer Gaia Price  342 non-null    float64
 3   Cost Material     342 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 10.8+ KB


In [59]:
def convert_to_minutes(time_string):
    # Split the time string into hours, minutes, and seconds
    try:
        hours, minutes, seconds = map(int, time_string.split(':'))
        # Calculate total minutes: hours * 60 + minutes + seconds / 60
        total_minutes = hours * 60 + minutes + seconds / 60
        return total_minutes
    except ValueError:
        return None  # Return None for invalid or NaN inputs

In [60]:
# convert column AU Time (Min) to float
item_code['AU Time (Min)'] = item_code['AU Time (Min)'].apply(convert_to_minutes)

In [64]:
item_code

,No,AU Time (Min),Offer Gaia Price,Cost Material
0,011,30.0,49.0,15
1,012,15.0,33.0,10
2,013,15.0,30.0,9
3,014,15.0,34.0,10
4,015,30.0,83.0,25
...,...,...,...,...
337,986,30.0,71.0,21
338,987,30.0,67.0,20
339,990,45.0,134.0,40
340,999,30.0,0.0,0


In [65]:
item_code.columns = ['item_number', 'duration', 'price', 'cost_material']

In [66]:
item_code.to_csv('cleaned_item_code.csv', index=False)

In [67]:
print(item_code)

    item_number  duration  price  cost_material
0           011      30.0   49.0             15
1           012      15.0   33.0             10
2           013      15.0   30.0              9
3           014      15.0   34.0             10
4           015      30.0   83.0             25
..          ...       ...    ...            ...
337         986      30.0   71.0             21
338         987      30.0   67.0             20
339         990      45.0  134.0             40
340         999      30.0    0.0              0
341        1Dsy      30.0   96.0             29

[342 rows x 4 columns]


#### Get Price

In [70]:
import pandas as pd
import ast  # To safely evaluate the string representation of lists in 'cleaned_item_numbers'

# Load the CSV files
item_code_df = pd.read_csv('cleaned_item_code.csv')
treatment_df = pd.read_csv('cleaned_treatment.csv')

# Create a dictionary from item_code_df for fast look-up of item prices
item_price_dict = dict(zip(item_code_df['item_number'].astype(str), item_code_df['price']))
item_cost_material_dict = dict(zip(item_code_df['item_number'].astype(str), item_code_df['cost_material']))
item_duration_dict = dict(zip(item_code_df['item_number'].astype(str), item_code_df['duration']))

# Function to calculate total price for each treatment based on cleaned_item_numbers
def calculate_total_price(cleaned_item_numbers):
    total_price = 0
    # Convert string representation of the list to an actual list
    item_codes = ast.literal_eval(cleaned_item_numbers)
    # Accumulate the prices from item_code_df
    for item_code in item_codes:
        total_price += item_price_dict.get(item_code, 0)  # Default to 0 if item_code is not found
    return total_price

def calculate_total_cost(cleaned_item_numbers):
    total_cost = 0
    # Convert string representation of the list to an actual list
    item_codes = ast.literal_eval(cleaned_item_numbers)
    # Accumulate the prices from item_code_df
    for item_code in item_codes:
        total_cost += item_cost_material_dict.get(item_code, 0)  # Default to 0 if item_code is not found
    return total_cost

def calculate_total_duration(cleaned_item_numbers):
    total_duration = 0
    # Convert string representation of the list to an actual list
    item_codes = ast.literal_eval(cleaned_item_numbers)
    # Accumulate the prices from item_code_df
    for item_code in item_codes:
        total_duration += item_duration_dict.get(item_code, 0)  # Default to 0 if item_code is not found
    return total_duration

# Apply the function to each row in the treatment_df
treatment_df['total_price'] = treatment_df['cleaned_item_numbers'].apply(calculate_total_price)
treatment_df['total_cost'] = treatment_df['cleaned_item_numbers'].apply(calculate_total_cost)
treatment_df['total_duration'] = treatment_df['cleaned_item_numbers'].apply(calculate_total_duration)

# Create a new DataFrame with Treatment and total_price columns
aggregated_df = treatment_df[['Treatment', 'total_price', 'total_cost', 'total_duration']]  

aggregated_df


,Treatment,total_price,total_cost,total_duration
0,Consultation,83.0,25,30.0
1,Consultation and TP presentation,100.0,30,45.0
2,Treatment Planning,178.0,54,95.0
3,Quick clean - scalling,83.0,25,30.0
4,Routine Clean - dental checkup,156.0,47,65.0
...,...,...,...,...
116,"2 Bridge treatment 1A,1P",1714.0,514,140.0
117,Provisional crown – per tooth,331.0,99,45.0
118,"Repair of crown, bridge or splint - indirect",205.0,62,45.0
119,"Repair of crown, bridge or splint – direct",148.0,44,30.0


In [85]:
aggregated_df.to_csv('treatment_with_details.csv', index=False)

#### Upselling Recommendation

In [1]:
import pandas as pd
import numpy as np

In [2]:
upsell = pd.read_csv('upselling_recommendation.csv')
print(upsell)

      No                                     Treatment  \
0      1                                  Consultation   
1      2              Consultation and TP presentation   
2      3                            Treatment Planning   
3      4                        Quick clean - scalling   
4      5                Routine Clean - dental checkup   
..   ...                                           ...   
116  117                     2 Bridge treatment  1A,1P   
117  118                 Provisional crown – per tooth   
118  119  Repair of crown, bridge or splint - indirect   
119  120    Repair of crown, bridge or splint – direct   
120  121                                       Monit 1   

                                                Upsell  
0                 Comprehensive Oral Health Assessment  
1        Advanced Treatment Plan with Smile Simulation  
2              Premium Treatment Strategy Consultation  
3                    Quick Clean + Fluoride Protection  
4       Full Oral 

In [3]:
upsell.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   No         121 non-null    int64 
 1   Treatment  121 non-null    object
 2   Upsell     120 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.0+ KB


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re

# Sample data - replace this with the actual CSV loading
treatment_df = pd.read_csv('upselling_recommendation.csv')

# Function to preprocess and tokenize the text into keywords
def preprocess(text):
    if pd.isna(text):  # Check if the value is NaN or missing
        return set()  # Return an empty set for missing values
    # Ensure the value is a string (in case it's not)
    text = str(text).lower()
    # Remove special characters (e.g., punctuation)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    # Split into words (tokens)
    tokens = text.split()
    # Remove stopwords (common words like "the", "and")
    tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS]
    # Return the tokens as a set (to handle unique keywords)
    return set(tokens)

# Function to calculate Jaccard similarity between two sets of tokens
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    if len(union) == 0:
        return 0
    return len(intersection) / len(union)

# Preprocess the Treatment column
treatment_df['treatment_keywords'] = treatment_df['Treatment'].apply(preprocess)

# Preprocess the Upsell column
treatment_df['upsell_keywords'] = treatment_df['Upsell'].apply(preprocess)

# Function to predict the most similar treatment for each upsell and calculate Jaccard score
def predict_upsell(upsell_keywords, current_treatment):
    best_match = None
    best_similarity = 0
    for index, row in treatment_df.iterrows():
        treatment_keywords = row['treatment_keywords']
        # Skip the current row (to avoid comparing a treatment with itself)
        if row['Treatment'] == current_treatment:
            continue
        similarity = jaccard_similarity(upsell_keywords, treatment_keywords)
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = row['Treatment']
    return best_match, best_similarity

# Apply the prediction function and extract both the predicted treatment and Jaccard score
treatment_df[['Predicted Upsell Treatment', 'Jaccard Score']] = treatment_df.apply(
    lambda row: pd.Series(predict_upsell(row['upsell_keywords'], row['Treatment'])),
    axis=1
)

# Output the updated DataFrame with the predicted upsell treatment and Jaccard score
treatment_df

# Save the new DataFrame to a CSV if needed
# treatment_df.to_csv('predicted_treatments_with_jaccard.csv', index=False)


,No,Treatment,Upsell,treatment_keywords,upsell_keywords,Predicted Upsell Treatment,Jaccard Score
0,1,Consultation,Comprehensive Oral Health Assessment,{consultation},"{oral, health, assessment, comprehensive}",Oral Health Index - Advance Hygiene (First Tim...,0.181818
1,2,Consultation and TP presentation,Advanced Treatment Plan with Smile Simulation,"{tp, presentation, consultation}","{simulation, treatment, advanced, plan, smile}",Treatment Planning,0.166667
2,3,Treatment Planning,Premium Treatment Strategy Consultation,"{planning, treatment}","{consultation, premium, treatment, strategy}",Consultation,0.250000
3,4,Quick clean - scalling,Quick Clean + Fluoride Protection,"{scalling, clean, quick}","{fluoride, clean, quick, protection}",Basic Clean,0.200000
4,5,Routine Clean - dental checkup,Full Oral Cancer Screening with Dental Checkup,"{routine, clean, dental, checkup}","{cancer, oral, dental, screening, checkup}",Dental Implant & Crown,0.142857
...,...,...,...,...,...,...,...
116,117,"2 Bridge treatment 1A,1P",Small Bridge Treatment with Enhanced Aesthetic...,"{1a1p, 2, treatment, bridge}","{treatment, enhanced, bridge, materials, small...",3 Bridge treatment,0.285714
117,118,Provisional crown – per tooth,Temporary Crown with Long-Term Upgrade Plan,"{provisional, crown, tooth}","{plan, temporary, upgrade, crown, longterm}",1 Crown,0.166667
118,119,"Repair of crown, bridge or splint - indirect",Crown/Bridge/Splint Repair with Premium Materi...,"{crown, bridge, indirect, splint, repair}","{upgrade, crownbridgesplint, material, premium...",Premium Aligner (Difficult Level),0.125000
119,120,"Repair of crown, bridge or splint – direct",Direct Repair with Free Follow-Up Adjustments,"{bridge, repair, crown, splint, direct}","{adjustments, free, followup, direct, repair}","Repair of crown, bridge or splint - indirect",0.111111


In [5]:
treatment_df.to_csv('predicted_upsell_recommendation.csv', index=False)

In [6]:
treatment_df.head(20)

,No,Treatment,Upsell,treatment_keywords,upsell_keywords,Predicted Upsell Treatment,Jaccard Score
0,1,Consultation,Comprehensive Oral Health Assessment,{consultation},"{oral, health, assessment, comprehensive}",Oral Health Index - Advance Hygiene (First Tim...,0.181818
1,2,Consultation and TP presentation,Advanced Treatment Plan with Smile Simulation,"{tp, presentation, consultation}","{simulation, treatment, advanced, plan, smile}",Treatment Planning,0.166667
2,3,Treatment Planning,Premium Treatment Strategy Consultation,"{planning, treatment}","{consultation, premium, treatment, strategy}",Consultation,0.250000
3,4,Quick clean - scalling,Quick Clean + Fluoride Protection,"{scalling, clean, quick}","{fluoride, clean, quick, protection}",Basic Clean,0.200000
4,5,Routine Clean - dental checkup,Full Oral Cancer Screening with Dental Checkup,"{routine, clean, dental, checkup}","{cancer, oral, dental, screening, checkup}",Dental Implant & Crown,0.142857
5,6,Advanced Hygiene,Laser-Assisted Deep Cleaning,"{hygiene, advanced}","{laserassisted, cleaning, deep}",Deep Gum treatment,0.200000
6,7,Basic Clean,Basic Clean + Whitening Treatment,"{clean, basic}","{clean, basic, treatment, whitening}",Treatment Planning,0.200000
7,8,Prevention Seal,Lifetime Preventive Sealant Package,"{prevention, seal}","{package, sealant, lifetime, preventive}",RC Quick Package - Front,0.166667
8,9,Caries Xrays,Full Dental Diagnostic Imaging,"{xrays, caries}","{diagnostic, imaging, dental}",Dental Implant & Crown,0.200000
9,10,Minor Fill front teeth,Cosmetic Composite Fillings,"{minor, teeth}","{cosmetic, composite, fillings}",NaN,0.000000
